In [1]:
import sys
!{sys.executable} -m pip install xgboost

In [2]:
# 📌 1. 라이브러리 임포트
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import gc
import os

In [3]:
# 📌 2. 메모리 최적화 함수 정의
def optimize_types(df):
    for col in df.select_dtypes(include=['float64']).columns:
        df[col] = pd.to_numeric(df[col], downcast='float')
    for col in df.select_dtypes(include=['int64']).columns:
        df[col] = pd.to_numeric(df[col], downcast='integer')
    return df

In [4]:
import pandas as pd

# ▶ 1. 조각 단위로 CSV 읽기
chunks = pd.read_csv(
    '병합/merged_final_train_셀_추가병합.csv',
    dtype={'최종카드론_신청경로코드': str},
    low_memory=False,
    chunksize=100000  # 필요시 더 작게 조절 가능 (예: 50000)
)

# ▶ 2. 조각을 이어 붙이기
train = pd.concat(chunks, ignore_index=True)

# ✅ 완료 메시지 출력
print(f"✅ CSV 병합 완료: {train.shape[0]}행 {train.shape[1]}열")


✅ CSV 병합 완료: 2400000행 763열


In [5]:
df_sample = pd.read_csv('병합/merged_final_train_셀_추가병합.csv', nrows=1)
print(df_sample.columns[308])

최종카드론_신청경로코드


In [6]:
# test = pd.read_csv('병합/전체 병합_test.csv')

In [7]:
train

,기준년월,ID,남녀구분코드,회원여부_이용가능,회원여부_이용가능_CA,회원여부_이용가능_카드론,소지여부_신용,소지카드수_유효_신용,소지카드수_이용가능_신용,입회일자_신용,...,금액_할부전환_R6M,금액_할부전환_R3M,신청건수_ATM_CA_B0,신청건수_ATM_CL_B0,승인거절건수_B0M,승인거절건수_한도초과_B0M,승인거절건수_BL_B0M,승인거절건수_BL_R3M,승인거절건수_입력오류_R3M,Segment.1
0,201807,TRAIN_000000,2,1,1,0,1,1,1,20130101,...,0,0,0,0,0,0,0,0,0,D
1,201807,TRAIN_000001,1,1,1,1,1,1,1,20170801,...,0,0,0,0,0,0,0,0,0,E
2,201807,TRAIN_000002,1,1,1,0,1,1,1,20080401,...,0,0,0,0,0,0,0,0,0,C
3,201807,TRAIN_000003,2,1,1,0,1,2,2,20160501,...,0,0,0,0,0,0,0,0,0,D
4,201807,TRAIN_000004,2,1,1,1,1,1,1,20180601,...,0,0,0,0,0,0,0,0,0,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,2,1,1,1,1,1,1,20010701,...,0,0,0,0,0,0,0,0,0,E
2399996,201812,TRAIN_399996,2,1,1,1,1,1,1,20170701,...,0,0,0,0,0,0,0,0,0,D
2399997,201812,TRAIN_399997,1,1,1,0,1,1,1,20090501,...,0,0,0,0,0,0,0,0,0,C
2399998,201812,TRAIN_399998,1,1,1,1,1,1,1,20130101,...,0,0,0,0,0,0,0,0,0,E


### Segment 중복이 있어서 제거

In [13]:
# Segment.1 → Segment 로 컬럼명 변경
train = train.rename(columns={'Segment.1': 'Segment'})

In [17]:
print(train.columns[train.columns.str.contains('Segment')])

Index(['Segment'], dtype='object')


In [19]:
# 'Segment'라는 이름의 컬럼이 여러 개일 경우 인덱스를 확인
segment_idx = [i for i, col in enumerate(train.columns) if col == 'Segment']
print(f"📌 Segment 컬럼 위치: {segment_idx}")

📌 Segment 컬럼 위치: [704]


In [16]:
# 중복된 컬럼 이름을 가진 열 중 첫 번째만 남기고 제거
_, idx = np.unique(train.columns, return_index=True)
train = train.iloc[:, sorted(idx)]

In [20]:
if 'Unnamed: 0' in train.columns:
    train = train.drop(columns=['Unnamed: 0'])

In [21]:
# ID 컬럼 제거
if 'ID' in train.columns:
    train = train.drop(columns=['ID'])

In [22]:
# 📌 4. 메모리 최적화 적용
train = optimize_types(train)
gc.collect()

190

In [23]:
# 📌 5. 피처/타깃 분리
X = train.drop(columns=['Segment'])
y = train['Segment']

import pickle

# ✅ X.columns 저장 (예측 시 컬럼 순서 일치용)
with open('XGBoost/X_columns.pkl', 'wb') as f:
    pickle.dump(X.columns.tolist(), f)

In [24]:
# 📌 6. object → category 변환
for col in X.select_dtypes(include='object').columns:
    X[col] = X[col].astype('category')

In [25]:
# 📌 7. 타깃 라벨 인코딩
le = LabelEncoder()
y_encoded = le.fit_transform(y)

In [26]:
print(le.classes_)

['A' 'B' 'C' 'D' 'E']


In [27]:
# 📌 8. 클래스 불균형 대응 가중치 계산
from collections import Counter
class_counts = Counter(y_encoded)
total = sum(class_counts.values())
weights = {cls: total / count for cls, count in class_counts.items()}
sample_weights = np.array([weights[c] for c in y_encoded])

In [28]:
# 📌 9. K-Fold 설정
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [30]:
import pickle
import gc

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y_encoded)):
    print(f"\n🚀 Fold {fold+1} 시작")

    # 학습 데이터 분할
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y_encoded[train_idx], y_encoded[val_idx]
    w_train = sample_weights[train_idx]

    # 모델 생성
    model = xgb.XGBClassifier(
        objective='multi:softprob',
        num_class=len(np.unique(y_encoded)),
        eval_metric='mlogloss',
        tree_method='gpu_hist',
        device='cuda',
        enable_categorical=True,
        random_state=42,
        verbosity=0
    )

    model.fit(X_train, y_train, sample_weight=w_train)

    # 평가
    preds_val = model.predict(X_val)
    f1 = f1_score(y_val, preds_val, average='micro')
    print(f"✅ Fold {fold+1} f1_micro: {f1:.4f}")

    # 중요도 저장
    importance_dict = model.get_booster().get_score(importance_type='gain')

    with open(f'XGBoost/importance_fold{fold}.pkl', 'wb') as f:
        pickle.dump(importance_dict, f)

    with open(f'XGBoost/f1_fold{fold}.txt', 'w') as f:
        f.write(str(f1))

    # 메모리 정리
    del model, X_train, X_val, y_train, y_val, w_train, preds_val, importance_dict
    gc.collect()


🚀 Fold 1 시작
✅ Fold 1 f1_micro: 0.8896

🚀 Fold 2 시작
✅ Fold 2 f1_micro: 0.8893

🚀 Fold 3 시작
✅ Fold 3 f1_micro: 0.8891

🚀 Fold 4 시작
✅ Fold 4 f1_micro: 0.8900

🚀 Fold 5 시작
✅ Fold 5 f1_micro: 0.8890


In [ ]:
10 / 0

In [31]:
fold = 3  # 가장 성능 좋은 Fold 기준 모델 저장(Fold 3 → index 2)

train_idx, val_idx = list(skf.split(X, y_encoded))[fold]
X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
y_train = y_encoded[train_idx]
w_train = sample_weights[train_idx]

model = xgb.XGBClassifier(
    objective='multi:softprob',
    num_class=len(np.unique(y_encoded)),
    eval_metric='mlogloss',
    tree_method='gpu_hist',
    device='cuda',
    enable_categorical=True,
    random_state=42,
    verbosity=0
)

model.fit(X_train, y_train, sample_weight=w_train)

# 저장
with open('XGBoost/final_model.pkl', 'wb') as f:
    pickle.dump(model, f)

print("✅ Fold 3 모델 저장 완료")

✅ Fold 3 모델 저장 완료


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_8604\3476234206.py:23: UserWarning: [15:35:03] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  pickle.dump(model, f)
